# Simulating Poisson processes and Brownian bridges

## Simulating an inhomogeneous Poisson process by thinning

#### Inhomogeneous Poisson process

The goal here is to experiment around the thinning of inhomogeneous Poisson processes. An \textit{inhomogeneous Poisson process} (or non-homogeneous Poisson process) is a stochastic process $ \{ N(t) \}_{t \geq 0}$ with a time-varying intensity function $ \lambda(t) $ such that:    
     - **Incremental Independence:** For any $ 0 \leq t_1 < t_2 < \dots < t_n $, the increments $ N(t_2) - N(t_1), N(t_3) - N(t_2), \dots, N(t_n) - N(t_{n-1}) $ are independent.  
     - **Inhomogeneous Poisson Increment Property:** The number of events occurring in an interval $ (s, t] $, for $ s < t $, follows a Poisson distribution with mean
    $$
    \mathbb{E}[N(t) - N(s)] = \int_s^t \lambda(u) \, du
    $$
    That is,
    $$
    N(t) - N(s) \sim \text{Poisson} \left( \int_s^t \lambda(u) \, du \right).
    $$   
    - **Initial Condition:** $N(0) = 0 $.  
    - **Interarrival time** (or event time): The probability density function $f_{T_1}(t_0,t)$ of the first arrival time $T_1$ after $t_0$ is: $$f_{T_1}(t)=\lambda(t)e^{-\Lambda(t)}, \text{ with }\Lambda(t)=\int_{t_0}^t \lambda(u) \, du.$$

#### The thinning method
An inhomogeneous Poisson process of rate $\lambda(t)\leq \lambda_M, \forall t$ can be simulated by considering a bound homogeneous Poisson process of rate $\lambda_M$:

- starting at $\tau_{0}=0$
- generating the interarrival time $\tau$ of the bound Poisson process of rate $\lambda_M$ by inversion sampling of the exponential law of parameter $\lambda_M$
- resampling this event as a true event with probability $\frac{\lambda(\tau+\tau_0)}{\lambda_M} $
- If $\tau+\tau_0$ is resampled as the interarrival time of the true Poisson process, output $\tau+\tau_0$. If not, update $\tau_0 = \tau_0 + \tau$ and repeat the previous steps.

Refs:

(1) APA Lewis, P. A., & Shedler, G. S. (1979). Simulation of nonhomogeneous Poisson processes by thinning. Naval Research Logistics Quarterly, 26(3), 403-413.

### Questions
1. Show that the thinning simulation method is correct. (Hint: Adapt exercise 5.1)

Library importation

In [1]:
import numpy as np
import random, math
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.lines as mlines
from matplotlib.ticker import FixedLocator

plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.minor.size'] = 8
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.minor.size'] = 8

We consider the problem of simulating from the first event time of a non-homogenous Poisson process of rate $\lambda$:
$$\lambda(t) = \max(\sin(t),0), \forall t\geq 0 $$

### Questions
2. Find $\lambda_M$ such that $\lambda_M\geq \lambda(t), \forall t$  
In the code below, please,
3. Assign a correct value to `self.lambda_M`
4. Code the `homogeneousPP_generation` routine which outputs the interarrival time of a Poisson process of homogeneous rate `self.lambda_M` by inversion sampling
5. Code the `thinning_generation` routine which outputs the interarrival time of the inhomogeneous Poisson process of rate $\lambda(t)$ (given by the routine `lambda_rate`) by thinning using the PP of rate $\lambda_M$
6. Code the `theoretical_arrival` routine which outputs the probability density of a given time to be the interarrival time of the target inhomogeneous PP


In [2]:
class max_cos_PP:
    def __init__(self,lambda_M):
        # Parameter initialization
        self.lambda_M = lambda_M
    
    def lambda_rate(self,t):
        return max(0,math.sin(t))
    
    def homogeneousPP_generation(self):
        # Write a routine generating the event times of the
        # homogeneous PP of rate lambda_M
        return  
        
    def direct_generation(self):
        # Routine generating by inversion the event times
        # of the PP of rate lambda    
        ran_tot = -math.log(random.uniform(0.0,1.0))
        tau = 2.0 * math.pi * (ran_tot//2)
        ran_tot = ran_tot % 2.0
        tau +=  math.acos(1-ran_tot)
        return tau
        
    def thinning_generation(self):
        # Write a routine generating the event times of the
        # PP of rate lambda by thinning
        return 
            
    def theoretical_arrival(self,t):
        # Write a routine generating the probability 
        # value for t to be the first arrival time 
        # of the target PP
        return 

### Questions
7. Compare the normed histograms of the interarrival times obtained with the `direct_generation`(directly sampling by inversion of the inhomogeneous PP), the `thinning_generation` and the theoretical value. Compare to the normed histogram of the interarrival times of the bound homogeneous PP of rate $\lambda_M$.

8. Compare the cumulative histograms of the first event times obtained with the `direct_generation`(directly sampling by inversion of the inhomogeneous PP), the `thinning_generation` and the theoretical values. Compare also to the cumulative histogram of the first event times of the bound homogeneous PP of rate $\lambda_M$. Confirm why the thinning method is correct.

9. **[BONUS]** How is the number of generated bound events required to resample a true one behaving with $\lambda_M$? Please, provide a plot.
10. **[BONUS]** What is a good choice for $\lambda_M$?

## Generating Brownian motion intermediate position

### Brownian bridge

We consider a sequence 
$$(B_{t_1},B_{t_2},\dots,B_{t_{n+1}}), \text{ with } t_1<t_2<\dots<t_{n+1}$$ 
of successive positions of a Brownian motion. In exercise 6.4, we saw how to sample the intermediate positions, i.e. $$(B_{t_1},\mathbf{B_{s_1}},B_{t_2},\mathbf{B_{s_2}},\dots,\mathbf{B_{s_n}},B_{t_{n+1}}), \text{ with } t_1<s_1<t_2<s_2<\dots<s_n<t_{n+1}.$$

For an intermediate position at time $s_k$, 
$$ B_{s_k} = \frac{t_{k+1}-s_k}{t_{k+1}-t_k}B_{t_k} + \frac{s_k - t_k}{t_{k+1}-t_k}B_{t_{k+1}} + \sqrt{t_{k+1}-t_k} Z,$$
with, $$Z\sim \mathcal{N}\left(0,\frac{(t_{k+1}-s_k)(s_k-t_k)}{t_{k+1}-t_k)^2}\right).
$$

### Questions
In the code below, please,

11. Code the `intermediate_position` routine which generates an intermediate position at a given time between two successive positions of the Brownian motions.
12. Code the `complete_motion` routine which generates the intermediate middle position between all two successive positions of a given Brownian motion sequence

In [6]:
class brownian_motion:
    def __init__(self,):
        # Parameter initialization
        pass
    
    def position_sequence(self, xinit, t_arr):
        # outputs a sequence of successive positions of 
        # a Brownian motion at times set by the array t_arr and starting 
        # from a given initial position xinit
        dt_list = t_arr[1:] - t_arr[:-1]
        # The Brownian increments are normally distributed 
        # with mean 0 and variance the time elapsed between 
        # two positions
        dB = np.sqrt(dt_list) * np.random.randn(len(t_arr)-1)
        B = np.insert(xinit + np.cumsum(dB),0,xinit)
        return B
    
    def intermediate_position(self, s, t1, t2, Bt1, Bt2):        
        # Write a routine generating the position of the Brownian
        # motion at time s when given its positions Bt1 and Bt2 
        # respectively at time t1 and t2
        return 
    
    def complete_motion(self, t_arr, Bt):     
        # Write a routine generating the intermediate middle-time 
        # position of the Brownian motion having as a sequence 
        # of positions Bt at times given by the sequence t_arr
        return 

## Questions
13. Generate the positions at time $0,1,2, \dots 10$ (time increment = 1) of a Brownian motion and complete the sequence of the positions of the sequence down to a time increment of 0.03125. Plot an example.
14. Check your result by comparing from samples obtained by direct simulation and obtained by the iterated generation:  
- the distribution of the position value at $t=5-0.03125$. Add the expected theoretical distribution to the plot.
- the distribution of the maximal absolute position value obtained. 
-  the distribution of the time $t$ at which the maximal absolute position value is reached.